In [1]:
from elasticsearch import Elasticsearch

In [4]:
es = Elasticsearch([{"host":"localhost","port":9200, "scheme": "http"}])
print(f"Elasticsearch instantiated? {es.ping()}")

Elasticsearch instantiated? True


In [17]:
##old configuration

configurations= {
    "settings" : {
        "number_of_shards": 1,
        "number_of_replicas": 1,
        "analysis": {
            "filter": {
                "english_stop": {
                    "type": "stop",
                    "stopwords_path": "my_stoplist.txt"
                }
            },
            "analyzer": {
                "stopped": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
                        "english_stop"
                    ]
                }
            }
      }
    },
    "mappings": {
        "properties": {
            "content": {
                "type": "text",
                "fielddata": True,
                "analyzer": "stopped",
                "index_options": "positions"
            }
        }
    }
}

In [18]:
## new configuration

configurations = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 1,
        "analysis": {
            "filter": {
                "english_stop": {
                    "type": "stop",
                    "stopwords_path": "my_stoplist.txt"
                },
                "english_stemmer": {
                    "type": "stemmer",
                    "language": "english"
                }
            },
            "analyzer": {
                "stopped_stemmed": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
                        "english_stop",
                        "english_stemmer"  # Adding the stemmer filter here
                    ]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {
                "type": "text",
                "fielddata": True,
                "analyzer": "stopped_stemmed",  # Use the new analyzer
                "index_options": "positions"
            }
        }
    }
}

In [21]:
from elasticsearch import exceptions

try:
    es.indices.create(index="my_index_1_2", body=configurations)
except exceptions.RequestError as e:
    print(f"RequestError: {e.info}")

/tmp/ipykernel_120179/2827981092.py:4: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index="my_index_1_2", body=configurations)


In [24]:
# Define your document data
document = {
    "somethingelse": "This is the content of an an an your document indexer."
}

# Index your document
index_name = "my_index_1"  # Replace with your desired index name
document_id = "AP003898900"  # Replace with a unique identifier for your document

# Index the document with your custom analyzer
es.index(index=index_name, id=document_id, document=document)

{'_index': 'my_index_1',
 '_type': '_doc',
 '_id': 'AP003898900',
 '_version': 3,
 'result': 'updated',
 '_shards': {'total': 2, 'successful': 2, 'failed': 0},
 '_seq_no': 2,
 '_primary_term': 1}